<a href="https://colab.research.google.com/github/segomezz/Practica2-IA/blob/main/Pr%C3%A1ctica_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Importación y preparación del dataset

El dataset Lending Club contiene información financiera de préstamos otorgados entre 2007 y 2018. Cuenta con más de 2 millones de registros y múltiples variables tanto numéricas como categóricas. La variable objetivo seleccionada es loan_status, que representa el estado final del préstamo y posee más de 10 categorías, de las cuales seleccionamos al menos 4 para abordar el problema como clasificación multiclase. Este conjunto de datos es adecuado para aplicar técnicas de aprendizaje supervisado y no supervisado, ya que permite evaluar el desempeño de los modelos ante clases desbalanceadas y perfiles de riesgo.

In [ ]:
import pandas as pd
import numpy as np
import zipfile

# Cargar el archivo zip directamente
file_path = '/content/drive/MyDrive/2025-I/Introducción a la inteligencia Artificial/Practica 2/accepted_2007_to_2018Q4.csv.gz'

df = pd.read_csv(file_path, low_memory=False, compression='gzip')

# Vista previa
print(df.shape)
df.head()

(2260701, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['loan_amnt'].plot(kind='hist', bins=20, title='loan_amnt')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['funded_amnt'].plot(kind='hist', bins=20, title='funded_amnt')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['term'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='index', y='term', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['grade'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='index', y='grade', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_27['sub_grade'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_27, x='index', y='sub_grade', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# Ver las clases posibles de loan_status
df['loan_status'].value_counts()

,count
loan_status,
Fully Paid,1076751
Current,878317
Charged Off,268559
Late (31-120 days),21467
In Grace Period,8436
Late (16-30 days),4349
Does not meet the credit policy. Status:Fully Paid,1988
Does not meet the credit policy. Status:Charged Off,761
Default,40


In [ ]:
# Filtrar las clases más frecuentes para convertir esto en clasificación multiclase
selected_statuses = ['Fully Paid', 'Charged Off', 'Late (31-120 days)', 'Default']
df = df[df['loan_status'].isin(selected_statuses)]

# Eliminar columnas con más del 50% de datos nulos
df = df.loc[:, df.isnull().mean() < 0.5]

# Seleccionar algunas variables numéricas y categóricas representativas
cat_vars = ['grade', 'home_ownership', 'verification_status', 'purpose', 'term']
num_vars = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'open_acc', 'revol_util']

df = df[cat_vars + num_vars + ['loan_status']].dropna()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

df_encoded = pd.get_dummies(df, columns=cat_vars)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_scaled = scaler.fit_transform(df_encoded[num_vars])
df_encoded[num_vars] = num_scaled

In [ ]:
from scipy.stats import zscore
import numpy as np
import pandas as pd

# 1. Calcular los z-scores solo de las variables numéricas
z_scores = np.abs(zscore(df_encoded[num_vars]))

# 2. Filtrar el dataset sin outliers (z-score < 3 en TODAS las columnas numéricas)
df_no_outliers = df_encoded[(z_scores < 3).all(axis=1)]

# 3. Determinar cuántas filas queremos como "outliers" (5% del dataset limpio)
n_outliers = int(0.05 * len(df_no_outliers))

# 4. Obtener las filas que fueron eliminadas (los verdaderos outliers detectados)
df_all_outliers = df_encoded[~df_encoded.index.isin(df_no_outliers.index)]

# 5. Tomar una muestra aleatoria de outliers (solo 5%)
# Verifica cuántos outliers realmente tienes disponibles
available_outliers = df_all_outliers.shape[0]
n_outliers_final = min(n_outliers, available_outliers)

# Toma solo los que puedas (sin reemplazo)
df_outliers_5pct = df_all_outliers.sample(n=n_outliers_final, random_state=42)

# 6. Combinar los datos limpios + los outliers seleccionados
df_5pct_outliers = pd.concat([df_no_outliers, df_outliers_5pct])

In [ ]:
from sklearn.utils import resample

# Dataset balanceado
min_class = df_no_outliers['loan_status'].value_counts().min()

dfs = []
for status in df_no_outliers['loan_status'].unique():
    df_class = df_no_outliers[df_no_outliers['loan_status'] == status]
    dfs.append(resample(df_class, replace=False, n_samples=min_class, random_state=42))

df_balanced = pd.concat(dfs)

# Dataset desbalanceado (sin modificación)
df_unbalanced = df_no_outliers.copy()

In [ ]:
datasets = {}

# Base sin escalado: deshacer el escalado en una copia
df_unscaled = df_encoded.copy()
df_unscaled[num_vars] = df[num_vars]  # recuperamos los valores originales sin escalar

### CASOS 1-4: sin escalado
datasets['1'] = df_no_outliers.copy()                        # No escalado, sin outliers, desbalanceado
datasets['2'] = df_balanced.copy()                           # No escalado, sin outliers, balanceado
datasets['3'] = df_5pct_outliers.copy()                      # No escalado, con 5% outliers, desbalanceado
datasets['4'] = pd.concat([
    df_balanced,
    df_5pct_outliers[df_5pct_outliers['loan_status'].isin(df_balanced['loan_status'].unique())]
]).drop_duplicates().copy()  # Balanceado con 5% outliers

### CASOS 5-8: con escalado (ya aplicado en df_encoded y derivados)
datasets['5'] = df_no_outliers.copy()                        # Escalado, sin outliers, desbalanceado
datasets['6'] = df_balanced.copy()                           # Escalado, sin outliers, balanceado
datasets['7'] = df_5pct_outliers.copy()                      # Escalado, con 5% outliers, desbalanceado
datasets['8'] = pd.concat([
    df_balanced,
    df_5pct_outliers[df_5pct_outliers['loan_status'].isin(df_balanced['loan_status'].unique())]
]).drop_duplicates().copy()  # Escalado, balanceado, con 5% outliers

# 2) Aprendizaje supervisado

# 3) Aprendizaje no supervisado